In [1]:
#Skripte laden
%reload_ext autoreload
%autoreload 2
import os
os.chdir('/home/safiul/email-classification')
import data
from Email_Classification.Functions.packages import *
import Email_Classification.Functions.Functions as fn
import Email_Classification.Functions.Helper as hp
import Email_Classification.Functions.parameters as par

In [2]:
param_process = par.params['param_process']
param_nlp = par.params['param_nlp']
param_lgb = par.params['param_lgb']
param_rf = par.params['param_rf']
param_nb = par.params['param_nb']

In [ ]:
#Pfade zur Speicherung der ML-Modelle festlegen
path_nb = "Email_Classification/Models/Nonsequential Models/Naive Bayes/tfidf"
path_lgb = 'Email_Classification/Models/Nonsequential Models/Light Gradient Boosting/tfidf'
path_lr = "Email_Classification/Models/Nonsequential Models/Logistic Regression/tfidf"
path_rf = "Email_Classification/Models/Nonsequential Models/Random Forest/tfidf/zip"

In [4]:
%%time
#Dateien laden
headers = pd.read_parquet(data.HEADERS_FILE)
bodies = pd.read_parquet(data.BODIES_FILE)
targets = features.zedsets.prepare_target_most_common(
    pd.read_parquet(data.TARGETS_FILE),
    target_column='target_category',
    num_most_common_class=100
)

#Der E-Mail-Datensatz mit den notwendigen Spalten erstellen
emails = fn.createDataFrameEmail(headers = headers, 
                                 bodies = bodies, 
                                 targets = targets, 
                                 sample_size = param_process['sample_size'], 
                                 seed = 125)
del headers, bodies, targets

CPU times: user 1min 14s, sys: 14.6 s, total: 1min 29s
Wall time: 1min 44s


In [5]:
#Der Datensatz auf Test- und Trainingsmenge aufteilen
X_train, X_test, y_train, y_test = \
model_selection.train_test_split(emails.emails, 
emails.target_category, test_size =  param_process['test_size']
                                 , random_state = 123)
#Konvertieren von kategorialen Labels in Zahlen
LE = preprocessing.LabelEncoder()
LE.fit(emails.target_category)
y_full = LE.transform(emails.target_category)
y_train = LE.transform(y_train)
y_test = LE.transform(y_test)
del emails

In [6]:
%%time
#Datenvorverarbeiten
TP = fn.Text_process(nchar = param_process['nchar'], 
                     rmDigits = param_process['rmDigits'], 
                     trans_lower= param_process['trans_lower'])
TP.fit(X_train)
X_train_TP = fn.multiprocess_array(TP, X_train)
X_test_TP = fn.multiprocess_array(TP, X_test)
X_train_TP = pd.concat(X_train_TP)
X_test_TP = pd.concat(X_test_TP)
del X_train, X_test

CPU times: user 18 s, sys: 25.3 s, total: 43.3 s
Wall time: 5min 15s


In [7]:
%%time
#Stemming
CSL = fn.Text_normalizer(stem_lemma = param_process['stem_lemma'])
CSL.fit(X_train_TP)
X_train_TN = fn.multiprocess_array(CSL, X_train_TP)
X_test_TN = fn.multiprocess_array(CSL, X_test_TP)
X_train_TN = pd.concat(X_train_TN)
X_test_TN = pd.concat(X_test_TN)
X_train_TN_ = X_train_TN[X_train_TN.apply(lambda x: len(x.split())) != 0]
X_test_TN_ = X_test_TN[X_test_TN.apply(lambda x: len(x.split())) != 0]
y_train = y_train[X_train_TN.apply(lambda x: len(x.split())) != 0]
y_test = y_test[X_test_TN.apply(lambda x: len(x.split())) != 0]
del X_train_TP, X_test_TP, X_train_TN, X_test_TN

CPU times: user 29.7 s, sys: 17.8 s, total: 47.5 s
Wall time: 22min 12s


In [12]:
%%time
#Feature-Repräsentation und Feature-Selektion
TF_tfidf = TfidfVectorizer(ngram_range = param_nlp['ngram_range'], 
                           max_features= 1000)
TF_tfidf.fit(X_train_TN_)
X_train_tfidf = fn.multiprocess_array(TF_tfidf, X_train_TN_)
X_test_tfidf = fn.multiprocess_array(TF_tfidf, X_test_TN_)
X_train_tfidf = sp.vstack(X_train_tfidf).toarray()
X_test_tfidf = sp.vstack(X_test_tfidf).toarray()

CPU times: user 2min 22s, sys: 29.2 s, total: 2min 51s
Wall time: 3min 2s


In [23]:
%%time
#Training und Speicherung von Multinomial-Naive-Bayes-Modell
NB = MultinomialNB(alpha= param_nb['alpha'], 
                   fit_prior=param_nb['fit_prior'])
NB.fit(X_train_tfidf, 
       y_train)
fn.writemodel(path_nb, "NB_100000", NB, makeZip= False)

CPU times: user 11.5 s, sys: 1.13 s, total: 12.6 s
Wall time: 12.6 s


MultinomialNB(alpha=0.01)

In [ ]:
%%time
#Training und Speicherung des Modells der logistischen Regression 
lr = LogisticRegression(penalty = 'l2',
                         max_iter= 1000,
                         C = 1.0,
                         fit_intercept = True,
                         tol = 1e-4,
                         solver = 'sag',
                         multi_class='multinomial',
                         n_jobs = -1,
                         verbose = 1,
                         random_state = 125)
lr.fit(X_train_tfidf, y_train)
fn.writemodel(path_lr, "lr_10000", 
              rf, makeZip= False)

In [ ]:
%%time
#Training und Speicherung des Modells der Random-Forest
rf = RandomForestClassifier(random_state= param_rf['random_state'],
                            criterion=param_rf['criterion'],
                            max_features=param_rf['max_features'],
                            n_estimators = param_rf['n_estimators'],
                            min_samples_split = param_rf['min_samples_split'],
                            max_samples = param_rf['max_samples'],
                            min_impurity_decrease = param_rf['min_impurity_decrease'],
                            max_depth = param_rf['max_depth'],
                            n_jobs = param_rf['n_jobs'],
                            verbose= param_rf['verbose']
                           )                                         
rf.fit(X_train_tfidf, y_train)
fn.writemodel(path_rf, "RF_2000", rf, makeZip= True)

In [ ]:
%%time
#Training und Speicherung des Modells der multinomialen Light-Gradient-Boosting 
param_lgb = par.params['param_lgb']
train_data = lightgbm.Dataset(X_train_tfidf, label=y_train)
test_data = lightgbm.Dataset(X_test_tfidf, label=y_test)
LGB = lightgbm.train(param_lgb['parameters'],
                       train_data,
                       valid_sets=test_data,
                       num_boost_round = param_lgb['num_boost_round'],
                       early_stopping_rounds= param_lgb['early_stopping_rounds'],
                      feval= param_lgb['feval']
                      )
fn.writemodel(path = path_lgb, filename = 'LGB_2000', 
              variable = LGB, makeZip= False)